In [1]:
#bring imports here
import json
import requests
import numpy as np
import pandas as pd
import os
import sys
from time import sleep

URL for the data: https://ilthermo.boulder.nist.gov/ILT2/ilsearch?cmp=&ncmp=0&year=&auth=&keyw=deep%20eutectic%20solvent&prp=ngXh

In [2]:
paper_url = "https://ilthermo.boulder.nist.gov/ILT2/ilsearch?cmp=&ncmp=0&year=&auth=&keyw=Deep%20eutectic%20solvent&prp=ngXh"
 
r = requests.get(paper_url)
header = r.json()['header']
papers = r.json()['res']
i = 1 
data_url = 'http://ilthermo.boulder.nist.gov/ILT2/ilset?set={paper_id}'
for paper in papers[:]: 
    
    r = requests.get(data_url.format(paper_id=paper[0]))
    data = r.json()['data']
    with open("/Users/jaime/Desktop/nist_data/speed_of_sound/%s.json" % i, "w") as outfile: #set destination path for files
        json.dump(r.json(), outfile)
    #then do whatever you want to data like writing to a file
    sleep(0.5) #import step to avoid getting banned by server
    i += 1

In [3]:
outer_old = pd.DataFrame()
outer_new = pd.DataFrame()

for i in range(12): #edit the number based on how many files were scraped
    with open("/Users/jaime/Desktop/nist_data/speed_of_sound/%s.json" % str(i+1)) as json_file:

        #grab data, data headers (names), the salt name
        json_full = json.load(json_file)                    
        json_data = pd.DataFrame(json_full['data'])
        json_datanames = np.array(json_full['dhead'])        # make names into array to add as columns headers for df
        json_data.columns =  json_datanames
        json_saltname = pd.DataFrame(json_full['components'])#components section contains names of DES components
        #print(json_saltname['name'])                #grabbing the HBD and HBA 

        inner_old = pd.DataFrame()
        inner_new = pd.DataFrame()

        #loop through the columns of the data, note that some of the 
        #json files are missing pressure data. 
        for indexer in range(len(json_data.columns)):        
            grab=json_data.columns[indexer]
            list = json_data[grab]
            my_list = [l[0] for l in list]
            dfmy_list = pd.DataFrame(my_list)
            dfmy_list.columns = [json_datanames[indexer][0]]
            inner_new = pd.concat([dfmy_list, inner_old], axis=1)
            inner_old = inner_new

        #print(inner_old.columns)


        #add the MW for HBD and HBA
        for i in range(len(json_saltname['name'])):
            if 'chloride' in json_saltname['name'][i] or 'bromide' in json_saltname['name'][i]:
                inner_old['HBA_MW']=json_saltname['mw'][i]
            else:
                inner_old['HBD_MW']=json_saltname['mw'][i]
                

        #add the DES components, i.e. HBA and HBD
        # they are not always listed in the same order on nist data, i.e., HBA always first. Will figure out later.
        for i in range(len(json_saltname['name'])):
            if 'chloride' in json_saltname['name'][i] or 'bromide' in json_saltname['name'][i]:
                inner_old['HBA']=json_saltname['name'][i]
            else:
                inner_old['HBD']=json_saltname['name'][i]
                
        #loop through the column names of the dataframe
        for j in range(len(inner_old.columns)):
            #if the words Mole fraction and a halogen are contained, values are correct and no value editing 
            #necessary and column is simply renamed to HBA mole fraction.
            if 'Mole fraction' in inner_old.columns[j] and 'chloride' in inner_old.columns[j] or 'Mole fraction' in inner_old.columns[j] and 'bromide' in inner_old.columns[j]:
                inner_old = inner_old.rename(columns={inner_old.columns[j]:'HBA Mole Fraction'})
            #if the words Mole Ratio and a halogen are contained, dataset was mislabeled but values are correct.
            #only need to rename column to HBA mole fraction.
            elif 'Mole ratio' in inner_old.columns[j] and 'chloride' in inner_old.columns[j] or 'Mole ratio' in inner_old.columns[j] and 'bromide' in inner_old.columns[j]:
                inner_old = inner_old.rename(columns={inner_old.columns[j]:'HBA Mole Fraction'})
            #if the words mole ratio are present, but no halogens, the ratio of the HBD is displayed and needs
            #to be changed to HBA mole fraction. First relabel the colum as HBA mole fraction.
            elif 'Mole ratio' in inner_old.columns[j] and not 'chloride' in inner_old.columns[j] or 'Mole ratio' in inner_old.columns[j] and not 'bromide' in inner_old.columns[j]:
                inner_old = inner_old.rename(columns={inner_old.columns[j]:'HBA Mole Fraction'})
                #apparently the numbers are strings so change to integer. May need to do this for every other column
                inner_old['HBA Mole Fraction'] = inner_old['HBA Mole Fraction'].astype(int)
                #next make an empty list that will hold all the new HBA mole fractions
                mole_fractions_list = []
                #loop through every HBD ratio in the column
                for k in range(len(inner_old['HBA Mole Fraction'])):
                    #Calculate the HBA mole fraction from every HBD ratio and append to the list
                    mole_fractions_list.append(1/(1+inner_old['HBA Mole Fraction'][k]))
                #finally make the list the new mole fraction column in the dataframe
                inner_old['HBA Mole Fraction'] = mole_fractions_list
            #in the last case, if the word mole fraction is present but not a halogen, HBD mole fraction is displayed.
            #Follow simialr process as before
            elif 'Mole fraction' in inner_old.columns[j] and not 'chloride' in inner_old.columns[j] or 'Mole fraction' in inner_old.columns[j] and not 'bromide' in inner_old.columns[j]:
                inner_old = inner_old.rename(columns={inner_old.columns[j]:'HBA Mole Fraction'})
                #convert to float instead since it is a decimal
                inner_old['HBA Mole Fraction'] = inner_old['HBA Mole Fraction'].astype(float)
                #empty list
                mole_fractions_list = []
                #loop through column
                for k in range(len(inner_old['HBA Mole Fraction'])):
                    #subtract 1 from HBD mole fraction to get HBA mole fraction and append to list
                    mole_fractions_list.append(1 - inner_old['HBA Mole Fraction'][k])
                #replace column   
                inner_old['HBA Mole Fraction'] = mole_fractions_list


        #add to the growing dataframe
        outer_new = pd.concat([inner_old, outer_old], axis = 0, ignore_index = True)
        outer_old = outer_new
    
outer_old.head(50)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,HBA,HBA Mole Fraction,HBA_MW,HBD,HBD_MW,"Pressure, kPa","Speed of sound, m/s","Temperature, K"
0,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1898.3,293.15
1,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1868.9,298.15
2,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1844.9,303.15
3,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1824.4,308.15
4,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1806.5,313.15
5,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1790.1,318.15
6,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1774.7,323.15
7,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1760.1,328.15
8,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1745.9,333.15
9,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1732.2,338.15


In [4]:
outer_old.dropna(inplace = True)

In [5]:
pd.DataFrame.to_csv(outer_old, path_or_buf='speed_of_sound.csv', index=False)

In [6]:
pd.read_csv('speed_of_sound.csv')

,HBA,HBA Mole Fraction,HBA_MW,HBD,HBD_MW,"Pressure, kPa","Speed of sound, m/s","Temperature, K"
0,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1898.3,293.15
1,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1868.9,298.15
2,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1844.9,303.15
3,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1824.4,308.15
4,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1806.5,313.15
5,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1790.1,318.15
6,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1774.7,323.15
7,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1760.1,328.15
8,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1745.9,333.15
9,benzyltripropylammonium chloride,0.25,269.86,glycerol,92.09,101,1732.2,338.15
